In [ ]:
# Generalized Newtonian problem on a unit square
# Poiseuille flow generated by a traction (pressure) difference
# Flow rate computation.

import numpy as np
import eztfem as ezt
from func import func
from traction_func import traction_func
from scipy.sparse.linalg import spsolve
from gn_elements import gn_elem

In [ ]:
# create mesh
mesh = ezt.quadrilateral2d([2, 10], 'tria6')

In [ ]:
# define the problem
elementdof = np.array([[2, 2, 2, 2, 2, 2],
                       [1, 0, 1, 0, 1, 0],
                       [1, 1, 1, 1, 1, 1]], dtype=int).transpose()
problem = ezt.Problem(mesh, elementdof, nphysq=2)
epsu = 1e-6
epsp = 1e-6
itermax = 100

In [ ]:
# Newton iterations
for iter in range(itermax):
    user = ezt.User()
    shape = 'triangle'
    user.xr, user.wg = ezt.gauss_legendre(shape, p=3)
    user.phi, user.dphi = ezt.basis_function(shape, 'P2', user.xr)
    user.psi, _ = ezt.basis_function(shape, 'P1', user.xr)
    user.coorsys = 0
    user.funcnr = 0
    user.func = func  # not used when funcnr == 0
    if iter == 0:
        user.mu = 1
        A, f = ezt.build_system(mesh, problem, ezt.stokes_elem, user)
    else:
        user.u = u
        user.gnmodel = 1  # 1: power-law, 2: Carreau, 3: Carreau-Yasuda
        user.m = 1
        user.n = 0.5
        user.eta0 = 1
        user.etainf = 1e-3
        user.llambda = 1
        user.a = 2
        A, f = ezt.build_system(mesh, problem, gn_elem, user)
    xr, user.wg = ezt.gauss_legendre('line', n=3)
    user.phi, user.dphi = ezt.basis_function('line', 'P2', xr)
    user.funcnr = 1
    user.func = traction_func
    ezt.add_boundary_elements(mesh, problem, f, ezt.stokes_natboun_curve,
                              user, physqrow=[0], curve=3)
    iess = ezt.define_essential(mesh, problem, 'curves', [0, 2], degfd=0)
    iess = ezt.define_essential(mesh, problem, 'curves', [0, 1, 2, 3],
                                degfd=1, iessp=iess)
    uess = np.zeros(problem.numdegfd)
    ezt.apply_essential(A, f, uess, iess)
    u = spsolve(A.tocsr(), f)
    if iter > 0:
        idx, _ = ezt.pos_array(problem, [i for i in range(mesh.nnodes)])
        delu = max(abs(u[idx[0]] - user.u[idx[0]])) / max(abs(u[idx[0]]))
        delp = max(abs(u[idx[1]] - user.u[idx[1]])) / max(abs(u[idx[1]]))
        print(f'iteration {iter}: delu {delu:.4e} delp {delp:.4e}')
        if delu < epsu and delp < epsp:
            print(f'Convergence at iteration {iter} \n')
            break
    else:
        print(f'iteration {iter}: Stokes solution')
else:
    raise RuntimeError('Iteration did not converge')

In [ ]:
# flow rate
user.u = u
flowrate = ezt.integrate_boundary_elements(mesh,
                                           problem,
                                           ezt.stokes_flowrate_curve,
                                           user, curve=1)
print('flowrate = ', flowrate)

In [ ]:
# Pressure in all nodes for plotting
xr = ezt.refcoor_nodal_points(mesh)
user.psi, _ = ezt.basis_function('triangle', 'P1', xr)
user.u = u
pressure = ezt.deriv_vector(mesh, problem, ezt.stokes_pressure, user)

In [ ]:
# derivatives of the velocity
xr = ezt.refcoor_nodal_points(mesh)
user.phi, user.dphi = ezt.basis_function('triangle', 'P2', xr)
user.u = u
user.comp = 0  # dudx
dudx = ezt.deriv_vector(mesh, problem, ezt.stokes_deriv, user)
user.comp = 1  # dudy
dudy = ezt.deriv_vector(mesh, problem, ezt.stokes_deriv, user)
user.comp = 2  # dvdx
dvdx = ezt.deriv_vector(mesh, problem, ezt.stokes_deriv, user)
user.comp = 3  # dvdy
dvdy = ezt.deriv_vector(mesh, problem, ezt.stokes_deriv, user)
user.comp = 4  # dvdx - dudy = vorticity
omega = ezt.deriv_vector(mesh, problem, ezt.stokes_deriv, user)
user.comp = 6  # divu, divergence of the velocity field
divu = ezt.deriv_vector(mesh, problem, ezt.stokes_deriv, user)
user.comp = 7  # gammadot, effective strain rate = sqrt(2II_D)
gammadot = ezt.deriv_vector(mesh, problem, ezt.stokes_deriv, user)


In [ ]:
# create a PyVista mesh for visualizing results
mesh_pv = ezt.generate_pyvista_mesh(mesh)

In [ ]:
# plot the x-velocity field
ezt.plot_sol(mesh_pv, problem, u, show_scalar_bar=True, n_colors=16,
             window_size=(800, 400), physq=0, degfd=0)

In [ ]:
# plot contourn of the x-velocity
ezt.plot_sol_contour(mesh_pv, problem, u, physq=0, degfd=0, nlevels=20)

In [ ]:
# quiver plot of the velocity field
ezt.plot_quiver(mesh_pv, problem, u, window_size=(800, 400), physq=0,
                scale=1.0)

In [ ]:
# plot the effective strain rate vector
ezt.plot_vector(mesh_pv, problem, gammadot)

In [ ]:
# contour plot of the effective strain rate
ezt.plot_vector_contours(mesh_pv, problem, gammadot, nlevels=8)

In [ ]:
# sample effective strain rate along a line
points = [[0.0, 0.3, 0.0], [1.0, 0.3, 0.0]]
_ = ezt.plot_vector_over_line(mesh_pv, problem, gammadot, points,
                              plot_mesh=False)